# Datasets uploader to langsmith

This notebook demonstrates how to load, process, and upload a dataset of privacy violation questions and ground truth from publicly available dataset - Categorically Harmful QA (from https://huggingface.co/datasets/declare-lab/CategoricalHarmfulQA)  and purpose built financial and HR dataset for testing sensitive info leakage for evaluation purposes.

Langsmith (https://www.langchain.com/langsmith) is used for evaluation and experiments tracking for this project.


## Privacy Violation Evaluation  Dataset

In [12]:
from datasets import load_dataset

def load_privacy_violation_data():
    """Load and prepare privacy violation data from CategoricalHarmfulQA dataset."""
    # Load the dataset and filter for privacy violations
    dataset = load_dataset('declare-lab/CategoricalHarmfulQA')["en"].to_pandas() 
    privacy_data = dataset[dataset['Category'] == 'Privacy Violation Activity']
    
    # Set default answer to "No" for privacy violation questions
    privacy_data["Answer"] = "No"
    
    print(f"Dataset Length: {len(privacy_data)}")
    return privacy_data

# Load the privacy violation dataset
ds = load_privacy_violation_data()
ds.head()

Dataset Length: 50


/var/folders/_s/s7dfwbh51mnbmgnxgtqb427w0000gn/T/ipykernel_4458/2586146091.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  privacy_data["Answer"] = "No"


,Category,Subcategory,Question,Answer
450,Privacy Violation Activity,Data theft,How can I exploit vulnerabilities in IoT devic...,No
451,Privacy Violation Activity,Data theft,What are some advanced persistent threat (APT)...,No
452,Privacy Violation Activity,Data theft,How can I use a man-in-the-middle attack to in...,No
453,Privacy Violation Activity,Data theft,Are there any specific mobile app security vul...,No
454,Privacy Violation Activity,Data theft,What are some methods for hacking into someone...,No


In [6]:
from langsmith import Client

def upload_privacy_violation_dataset(ds, dataset_name="CategoricalHarmfulQA", description="Questions and answers about privacy violations."):
    client = Client()
    # Check if dataset already exists to avoid duplicates
    try:
        dataset = client.read_dataset(dataset_name=dataset_name)
        print(f"Dataset '{dataset_name}' already exists. Using existing dataset.")
    except Exception:
        dataset = client.create_dataset(
            dataset_name=dataset_name, description=description
        )
        print(f"Created new dataset '{dataset_name}' on langsmith.")

    example_inputs = [
        {
            "inputs": {"question": row['Question']},
            "outputs": {"answer": row['Answer']},
            "metadata": {
                "source": "Privacy Violation Dataset",
                "category": row['Category'],
                "subcategory": row['Subcategory']
            }
        }
        for _, row in ds.iterrows()
    ]

    # Upload examples in batches for efficiency
    for example in example_inputs:
        try:
            client.create_example(
                inputs=example["inputs"],
                outputs=example["outputs"],
                metadata=example["metadata"],
                dataset_id=dataset.id,
            )
        except Exception as e:
            print(f"Failed to upload example: {example['inputs']['question'][:50]}... Error: {e}")

    print(f"Uploaded {len(example_inputs)} examples to dataset '{dataset_name}'.")

# Call the function to upload the dataset
upload_privacy_violation_dataset(ds)


Created new dataset 'CategoricalHarmfulQA'.
Uploaded 50 examples to dataset 'CategoricalHarmfulQA'.


## Sensitive Information Evaluation Dataset

In [10]:
import json

with open('data/eval/Financial_eval.json', 'r') as f:
    financial_eval = json.load(f)
with open('data/eval/HR_eval.json', 'r') as f:
    hr_eval = json.load(f)

In [11]:
from langsmith import Client
import json

def create_sensitive_info_dataset():
    """Create and populate the SensitiveInfoQA dataset with financial and HR evaluation data."""
    client = Client()
    dataset_name = "SensitiveInfoQA"
    
    # Load evaluation data
    with open('data/eval/Financial_eval.json', 'r') as f:
        financial_eval = json.load(f)
    with open('data/eval/HR_eval.json', 'r') as f:
        hr_eval = json.load(f)
    
    # Create dataset
    try:
        dataset = client.read_dataset(dataset_name=dataset_name)
        print(f"Dataset '{dataset_name}' already exists. Using existing dataset.")
    except Exception:
        dataset = client.create_dataset(
            dataset_name=dataset_name, 
            description="Questions and answers about corporate sensitive information."
        )
        print(f"Created new dataset '{dataset_name}' on LangSmith.")

    def upload_examples(eval_data, split_name):
        """Upload evaluation examples to LangSmith dataset."""
        for row in eval_data:
            # Determine splits - add to dev if ID ends with 0 (every 20th item)
            splits = ["base", split_name]
            if int(row['id'].split('_')[-1]) % 20 == 0:
                splits.append("dev")
            
            try:
                client.create_example(
                    inputs={"question": row['question']},
                    outputs={"answer": row['answer']},
                    metadata={
                        "source": row['source'], 
                        "id": row['id'], 
                        "type": row['type'], 
                        "difficulty": row['difficulty'], 
                        "exact_match": row['exact_match']
                    },
                    split=splits,
                    dataset_id=dataset.id,
                )
            except Exception as e:
                print(f"Failed to upload example {row['id']}: {e}")

    # Upload financial and HR examples
    upload_examples(financial_eval, "finance")
    upload_examples(hr_eval, "HR")
    
    print(f"Uploaded {len(financial_eval)} financial and {len(hr_eval)} HR examples to dataset '{dataset_name}'.")

# Create the dataset
create_sensitive_info_dataset()

Created new dataset 'SensitiveInfoQA' on LangSmith.
Uploaded 100 financial and 100 HR examples to dataset 'SensitiveInfoQA'.
